In [9]:
import argparse
import os
import logging

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable

import utils
import data_loader

parser = argparse.ArgumentParser()
parser.add_argument('--data_dir', default='./data/test',
                    help="Directory containing the dataset")
parser.add_argument('--model', type=str, required=True,
                    help="The model you want to test")
parser.add_argument('--weights', required=True,
                    help="The weights file you want to test")
parser.add_argument('--batch_size', default=256,
                    help="batch size")
parser.add_argument('--gpu', action='store_true', default='False',
                    help="GPU available")


def evaluate(model, loss_fn, dataloader):
    """ Evaluate the model on `num_steps` batches
    Args:
        model : (torch.nn.Module) model
        dataloader : (DataLoader) a torch.utils.data.DataLoader object that fetches training data
        num_steps : (int) # of batches to train on, each of size args.batch_size
    """

    # set model to test mode
    model.eval()

    model_dir = './results/' + model_name

    total_loss = 0.0
    total_correct = 0.0

    for i, (test_batch, labels_batch) in enumerate(dataloader):
        # move to GPU if available
        if args.gpu:
            test_batch, labels_batch = test_batch.cuda(), labels_batch.cuda()

        # convert to torch Variable
        test_batch, labels_batch = Variable(test_batch), Variable(labels_batch)

        # compute model output and loss
        output_batch = model(test_batch)
        loss = loss_fn(output_batch, labels_batch)

        total_loss += loss.item()
        acc = utils.accuracy(output_batch.data.cpu().numpy(), labels_batch.data.cpu().numpy())
        total_correct += acc

    print("Loss:{:.4f}\t Test Accuracy:{:.4f}".format(
        total_loss/len(dataloader),
        100 * total_correct / len(dataloader)
    ))

if __name__ == '__main__':

    # Load the parameters from parser
    args = parser.parse_args()

    model_name = args.model
    weights_path = args.weights
    batch_size = args.batch_size

    logging.info("Loading the test dataset...")

    # fetch train dataloader
    test_dataloader = data_loader.test_data_loader()

    logging.info("- done.")

    # Define the model and optimizer
    model = utils.get_network(args)
    checkpoint = torch.load(weights_path)
    model.load_state_dict(checkpoint['state_dict'])

    # fetch loss function
    loss_fn = nn.CrossEntropyLoss()

    # Train the model
    logging.info("Starting Test ...")
    evaluate(model, loss_fn, test_dataloader)

usage: ipykernel_launcher.py [-h] [--data_dir DATA_DIR] --model MODEL
                             --weights WEIGHTS [--batch_size BATCH_SIZE]
                             [--gpu]
ipykernel_launcher.py: error: the following arguments are required: --model, --weights


SystemExit: 2

In [2]:
import sys
import os
import shutil

import numpy as np
import torch
import torch.optim as optim

def get_network(args):
    """ Return the given network
    Args:
        args : (argparser)
    """

    if args.model == 'alexnet':
        from models.alexnet import alexnet
        net = alexnet()
    if args.model == 'zfnet':
        from models.ZFNet import ZFNet
        net = ZFNet()
    # elif args.net == 'vgg':
    #     from models.vgg import vgg
    #     net = vgg()
    # elif ...
    #       ...

    else:
        print('the network name you have entered is not supported yet')
        sys.exit()

    if args.gpu:
        net = net.cuda()

    return net

def get_optimizer(model_name, model, lr):
    """ Return the optimizer
    """

    if model_name == 'alexnet':
        optimizer = optim.Adam(model.parameters(), lr=lr)
    elif model_name == 'zfnet':
        optimizer = optim.SGD(model.parameters(), lr=lr, momentum = 0.9)
    # elif args.net == 'vgg':
    #     from models.vgg import vgg
    #     net = vgg()
    # elif ...
    #       ...

    else:
        print('the network name you have entered is not supported yet')
        sys.exit()

    return optimizer

def accuracy(outputs, labels):
    """ Compute the accuracy
    Args:
        outputs : (nd.ndarray) prediction
        labels : (nd.ndarray) real
    Returns:
          (float) accuaracy in [0,1]
    """
    outputs = np.argmax(outputs, axis=1)
    return np.sum(outputs==labels)/float(labels.size)

def save_checkpoints(state, is_best, checkpoint):
    """ Saves model and training parameters at checkpoint + 'last.pth.
    If is_best==True, also saves checkpoint + 'best.pth'

    Args:
        state : (dict) contains model's state_dict
        is_best : (bool) True if it is the best model
        checkpoint : (string) folder where parameters are to be saved
    """
    filepath = os.path.join(checkpoint, 'last.pth')
    if not os.path.exists(checkpoint):
        print("Checkpoint Direcotry doesn't exist. Making directory {}".format(checkpoint))
        os.mkdir(checkpoint)
    else:
        pass
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(checkpoint, 'best.pth'))

In [8]:
import argparse
import os
import logging

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable

import utils
import data_loader

parser = argparse.ArgumentParser()
parser.add_argument('--data_dir', default='./data/train',
                    help="Directory containing the dataset")
parser.add_argument('--model', type=str, required=True,
                    help="The model you want to train")
parser.add_argument('--lr', type=float, default=0.001,
                    help="Learning rate")
parser.add_argument('--epoch', type=int, default=50,
                    help="Total training epochs")
parser.add_argument('--batch_size', type=int, default=256,
                    help="batch size")
parser.add_argument('--gpu', action='store_true', default='False',
                    help="GPU available")


def train(model, optimizer, loss_fn, dataloader):
    """ Train the model on `num_steps` batches
    Args:
        model : (torch.nn.Module) model
        optimizer : (torch.optim) optimizer for parameters of model
        loss_fn : (string) a function that takes batch_output and batch_labels and computes the loss for the batch
        dataloader : (DataLoader) a torch.utils.data.DataLoader object that fetches training data
        num_steps : (int) # of batches to train on, each of size args.batch_size
    """

    # set model to training mode
    model.train()

    model_dir = './results/' + model_name
    best_acc = 0.0

    for epoch in range(epochs):
        epoch_loss = 0.0
        epoch_correct = 0.0

        for i, (train_batch, labels_batch) in enumerate(dataloader):
            # move to GPU if available
            if args.gpu:
                train_batch, labels_batch = train_batch.cuda(), labels_batch.cuda()

            # convert to torch Variable
            train_batch, labels_batch = Variable(train_batch), Variable(labels_batch)

            # compute model output and loss
            output_batch = model(train_batch)
            loss = loss_fn(output_batch, labels_batch)


            # clear previous gradients, compute gradients of all variables wrt loss
            optimizer.zero_grad()
            loss.backward()

            # performs updates using calculated gradients
            optimizer.step()

            epoch_loss += loss.item()
            acc = utils.accuracy(output_batch.data.cpu().numpy(), labels_batch.data.cpu().numpy())
            epoch_correct += acc

#             print("Epoch [{}]\t Batch [{}/{}]\t Loss:{:.4f}\t Accuracy:{:.4f}".format(epoch+1, i, len(dataloader), loss.item(), acc))

        print("Epoch [{}/{}]\t Loss:{:.4f}\t Accuracy:{:.4f}%".format(
            epoch + 1,
            epochs,
            epoch_loss/len(dataloader),
            100 * epoch_correct / len(dataloader)
        ))

        is_best = acc >= best_acc
        if is_best:
            logging.info("- Found new best accuracy")
            best_acc = acc

        utils.save_checkpoints(
            {'epoch': i + 1,
             'state_dict': model.state_dict(),
             'optim_dict': optimizer.state_dict()},
            is_best=is_best,
            checkpoint=model_dir
        )

if __name__ == '__main__':

    # Load the parameters from parser
    args = parser.parse_args()

    model_name = args.model
    lr = args.lr
    epochs = args.epoch
    batch_size = args.batch_size

    logging.info("Loading the training dataset...")

    # fetch train dataloader
    train_dataloader = data_loader.train_data_loader()

    logging.info("- done.")

    # Define the model and optimizer
    model = utils.get_network(args)
    optimizer = utils.get_optimizer(model_name, model, lr)

    # fetch loss function
    loss_fn = nn.CrossEntropyLoss()

    # Train the model
    logging.info("Starting training for {} epoch(s).".format(epochs))
    train(model, optimizer, loss_fn, train_dataloader)

usage: ipykernel_launcher.py [-h] [--data_dir DATA_DIR] --model MODEL
                             [--lr LR] [--epoch EPOCH]
                             [--batch_size BATCH_SIZE] [--gpu]
ipykernel_launcher.py: error: the following arguments are required: --model


SystemExit: 2

In [4]:
import os

import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets


def train_data_loader(batch_size=256, workers=1, shuffle=True):
    """ return training, test dataloader
    Args:
        batch_size : (int) dataloader batchsize
        workers : (int) # of subprocesses
        shuffle : (bool) data shuffle at every epoch
    Returns:
        train_data_loader : torch dataloader obj.
        test_data_loader : torch dataloader obj.
    """

    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean = [0.4913997551666284, 0.48215855929893703, 0.4465309133731618],
                             std = [0.24703225141799082, 0.24348516474564, 0.26158783926049628])
    ])

    train_dataset = datasets.CIFAR10(root='./data/train', train=True, download=True, transform=transform)
    train_data_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        num_workers=workers
    )

    return train_data_loader

def test_data_loader(batch_size=256, workers=1, shuffle=True):
    """ return training, test dataloader
    Args:
        batch_size : (int) dataloader batchsize
        workers : (int) # of subprocesses
        shuffle : (bool) data shuffle at every epoch
    Returns:
        train_data_loader : torch dataloader obj.
        test_data_loader : torch dataloader obj.
    """

    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean = [0.4913997551666284, 0.48215855929893703, 0.4465309133731618],
                             std = [0.24703225141799082, 0.24348516474564, 0.26158783926049628])
    ])

    test_dataset = datasets.CIFAR10(root='./data/test', train=False, download=True, transform=transform)
    test_data_loader = torch.utils.data.DataLoader(
        test_dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        num_workers=workers
    )

    return test_data_loader

In [7]:
import os
import torch
import torch.nn as nn


class ZFNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ZFNet, self).__init__()

        self.features = nn.Sequential(
            # layer 1
            nn.Conv2d(3, 96, kernel_size=7, stride=2, padding=1),
            nn.ReLU(inplace=True),
            # Local contrast norm.이 있어야 하는데 파이토치에는 해당 클래스가 없는 듯? LocalResponseNorm 과는 다른 건가?
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1, return_indices=True), # return_indices – if True, will return the max indices along with the outputs. Useful for torch.nn.MaxUnpool2d later

            # layer 2
            nn.Conv2d(96, 256, kernel_size=5, stride=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1, return_indices=True),

            # layer 3
            nn.Conv2d(256, 384, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),

            # layer 4
            nn.Conv2d(384, 384, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),

            # layer 5
            nn.Conv2d(384, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2, return_indices=True),
        )
        self.classifier = nn.Sequential(
            # layer 6
            nn.Linear(9216, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),

            # layer 7
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),

            nn.Linear(4096, num_classes),
        )

        self.feature_outputs = [0]*len(self.features)
        self.switch_indices = dict()
        self.sizes = dict()

    def forward(self, x):

        for i, layer in enumerate(self.features):
            if isinstance(layer, nn.MaxPool2d):
                x, indices = layer(x)
                self.feature_outputs[i] = x
                self.switch_indices[i] = indices
            else:
                x = layer(x)
                self.feature_outputs[i] = x

        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

def alexnet(**kwargs):
    model = ZFNet(**kwargs)
    return model